<a href="https://colab.research.google.com/github/KaifAhmad1/AB-Testing/blob/master/SolarPack_Assessment_2nd_Part_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Solar Panel Tilt Angle Optimization Using Machine Learning Model**




In [3]:
import numpy as np
import pandas as pd

In [19]:
radiation_file = '/content/drive/MyDrive/Solar Panel Tilt Angle Optimization/SolarRadiationData.csv'
solar_angle_file = '/content/drive/MyDrive/Solar Panel Tilt Angle Optimization/solar_angles_dataset.csv'
solar_forecasting_file = '/content/drive/MyDrive/Solar Panel Tilt Angle Optimization/Pasion et al dataset.csv'
file_2017to2019 = '/content/drive/MyDrive/Solar Panel Tilt Angle Optimization/2017_2019.csv'

In [20]:
radiation_data = pd.read_csv(radiation_file)

In [30]:
radiation_data

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [36]:
radiation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   UNIXTime                32686 non-null  int64         
 1   Data                    32686 non-null  object        
 2   Time                    32686 non-null  object        
 3   Radiation               32686 non-null  float64       
 4   Temperature             32686 non-null  int64         
 5   Pressure                32686 non-null  float64       
 6   Humidity                32686 non-null  int64         
 7   WindDirection(Degrees)  32686 non-null  float64       
 8   Speed                   32686 non-null  float64       
 9   TimeSunRise             32686 non-null  object        
 10  TimeSunSet              32686 non-null  object        
 11  DateTime                32686 non-null  datetime64[ns]
 12  Hour                    32686 non-null  int64 

In [31]:
radiation_data['DateTime'] = pd.to_datetime(radiation_data['Data'] + ' ' + radiation_data['Time'])
# Extract time component
radiation_data['Hour'] = radiation_data['DateTime'].dt.hour
radiation_data['Minute'] = radiation_data['DateTime'].dt.minute
radiation_data['Second'] = radiation_data['DateTime'].dt.second

# Print the modified dataset
print(radiation_data.head())

     UNIXTime                   Data      Time  Radiation  Temperature  \
0  1475229326  9/29/2016 12:00:00 AM  23:55:26       1.21           48   
1  1475229023  9/29/2016 12:00:00 AM  23:50:23       1.21           48   
2  1475228726  9/29/2016 12:00:00 AM  23:45:26       1.23           48   
3  1475228421  9/29/2016 12:00:00 AM  23:40:21       1.21           48   
4  1475228124  9/29/2016 12:00:00 AM  23:35:24       1.17           48   

   Pressure  Humidity  WindDirection(Degrees)  Speed TimeSunRise TimeSunSet  \
0     30.46        59                  177.39   5.62    06:13:00   18:13:00   
1     30.46        58                  176.78   3.37    06:13:00   18:13:00   
2     30.46        57                  158.75   3.37    06:13:00   18:13:00   
3     30.46        60                  137.71   3.37    06:13:00   18:13:00   
4     30.46        62                  104.95   5.62    06:13:00   18:13:00   

             DateTime  Hour  Minute  Second  
0 2016-09-29 23:55:26    23      5

In [39]:
print("Radiation Data Datetime Range:")
print(radiation_data['DateTime'].min(), "to", radiation_data['DateTime'].max())

Radiation Data Datetime Range:
2016-09-01 00:00:08 to 2016-12-31 23:55:01


In [22]:
solar_angle_data = pd.read_csv(solar_angle_file)

In [23]:
solar_angle_data

,Unnamed: 0,Date (M/D/YYYY),Time (H:MM:SS),Topocentric zenith angle,Top. azimuth angle (eastward from N),Top. azimuth angle (westward from S),Julian day
0,0,1/1/2017,0:00:00,157.863107,308.657327,128.657327,2.457754e+06
1,1,1/1/2017,0:15:00,160.034217,316.136781,136.136781,2.457754e+06
2,2,1/1/2017,0:30:00,161.901608,325.052191,145.052191,2.457754e+06
3,3,1/1/2017,0:45:00,163.360197,335.548911,155.548911,2.457754e+06
4,4,1/1/2017,1:00:00,164.293788,347.499018,167.499018,2.457754e+06
...,...,...,...,...,...,...,...
105211,105211,1/1/2020,22:45:00,144.648900,284.820719,104.820719,2.458850e+06
105212,105212,1/1/2020,23:00:00,147.463622,288.398876,108.398876,2.458850e+06
105213,105213,1/1/2020,23:15:00,150.216607,292.424007,112.424007,2.458850e+06
105214,105214,1/1/2020,23:30:00,152.884991,297.022677,117.022677,2.458850e+06


In [34]:
solar_angle_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105216 entries, 0 to 105215
Data columns (total 7 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Unnamed: 0                            105216 non-null  int64  
 1   Date                                  105216 non-null  object 
 2   Time                                  105216 non-null  object 
 3   Topocentric zenith angle              105216 non-null  float64
 4   Top. azimuth angle (eastward from N)  105216 non-null  float64
 5   Top. azimuth angle (westward from S)  105216 non-null  float64
 6   Julian day                            105216 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 5.6+ MB


In [35]:
# Convert 'Date' and 'Time' columns to datetime
solar_angle_data['DateTime'] = pd.to_datetime(solar_angle_data['Date'] + ' ' + solar_angle_data['Time'])

# Extract time component
solar_angle_data['Hour'] = solar_angle_data['DateTime'].dt.hour
solar_angle_data['Minute'] = solar_angle_data['DateTime'].dt.minute
solar_angle_data['Second'] = solar_angle_data['DateTime'].dt.second

# Drop the original 'Date' and 'Time' columns
solar_angle_data.drop(columns=['Date', 'Time'], inplace=True)

# Print the modified dataset
print(solar_angle_data.head())

   Unnamed: 0  Topocentric zenith angle  Top. azimuth angle (eastward from N)  \
0           0                157.863107                            308.657327   
1           1                160.034217                            316.136781   
2           2                161.901608                            325.052191   
3           3                163.360197                            335.548911   
4           4                164.293788                            347.499018   

   Top. azimuth angle (westward from S)    Julian day            DateTime  \
0                            128.657327  2.457754e+06 2017-01-01 00:00:00   
1                            136.136781  2.457754e+06 2017-01-01 00:15:00   
2                            145.052191  2.457754e+06 2017-01-01 00:30:00   
3                            155.548911  2.457754e+06 2017-01-01 00:45:00   
4                            167.499018  2.457754e+06 2017-01-01 01:00:00   

   Hour  Minute  Second  
0     0       0       0 

In [40]:
print("\nSolar Angle Data Datetime Range:")
print(solar_angle_data['DateTime'].min(), "to", solar_angle_data['DateTime'].max())


Solar Angle Data Datetime Range:
2017-01-01 00:00:00 to 2020-01-01 23:45:00


In [41]:
solar_forecasting_data = pd.read_csv(solar_forecasting_file)

In [42]:
solar_forecasting_data

,Location,Date,Time,Latitude,Longitude,Altitude,YRMODAHRMI,Month,Hour,Season,Humidity,AmbientTemp,PolyPwr,Wind.Speed,Visibility,Pressure,Cloud.Ceiling
0,Camp Murray,20171203,1145,47.11,-122.57,84,2.017120e+11,12,11,Winter,81.71997,12.86919,2.42769,5,10.0,1010.6,722
1,Camp Murray,20171203,1315,47.11,-122.57,84,2.017120e+11,12,13,Winter,96.64917,9.66415,2.46273,0,10.0,1011.3,23
2,Camp Murray,20171203,1330,47.11,-122.57,84,2.017120e+11,12,13,Winter,93.61572,15.44983,4.46836,5,10.0,1011.6,32
3,Camp Murray,20171204,1230,47.11,-122.57,84,2.017120e+11,12,12,Winter,77.21558,10.36659,1.65364,5,2.0,1024.4,6
4,Camp Murray,20171204,1415,47.11,-122.57,84,2.017120e+11,12,14,Winter,54.80347,16.85471,6.57939,3,3.0,1023.7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21040,USAFA,20180928,1530,38.95,-104.83,1947,2.018090e+11,9,15,Fall,11.66992,43.22510,9.79611,14,10.0,802.3,722
21041,USAFA,20180929,1300,38.95,-104.83,1947,2.018090e+11,9,13,Fall,18.22510,28.98247,10.88992,13,10.0,799.2,722
21042,USAFA,20180929,1400,38.95,-104.83,1947,2.018090e+11,9,14,Fall,15.52124,33.49167,8.24479,10,10.0,798.4,722
21043,USAFA,20180929,1500,38.95,-104.83,1947,2.018090e+11,9,15,Fall,6.63452,51.62163,12.47328,10,10.0,797.8,722


In [43]:
data_2017_to_2019 = pd.read_csv(file_2017to2019)

In [27]:
data_2017_to_2019

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,DHI,DNI,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Wind Speed,Unnamed: 18
0,2017,1,1,0,0,-0.6,0,0,0,-2.6,0,0,0,86.29,152.92,0.12,1003,1.7,NaN
1,2017,1,1,0,15,-0.6,0,0,0,-2.6,0,0,0,86.29,150.22,0.12,1003,1.7,NaN
2,2017,1,1,0,30,-0.6,0,0,0,-2.6,0,0,0,86.29,147.43,0.12,1003,1.7,NaN
3,2017,1,1,0,45,-0.6,0,0,0,-2.7,0,0,0,85.54,144.60,0.12,1003,1.6,NaN
4,2017,1,1,1,0,-0.7,0,0,0,-2.7,0,0,0,86.17,141.71,0.12,1003,1.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,2019,12,31,22,45,4.0,0,0,0,4.0,0,0,0,100.00,163.63,0.12,999,2.0,NaN
105116,2019,12,31,23,0,3.9,0,0,0,3.9,0,0,0,100.00,162.09,0.12,999,2.0,NaN
105117,2019,12,31,23,15,3.9,0,0,0,3.9,0,0,0,100.00,160.16,0.12,999,2.0,NaN
105118,2019,12,31,23,30,3.9,0,0,0,3.9,0,0,0,100.00,157.93,0.12,999,2.0,NaN


In [ ]:
https://chat.openai.com/share/b5dbacc1-d063-40de-a00a-b7444958f55c

In [ ]:
https://chat.openai.com/share/e8c7c656-45a6-484a-9ef8-8256432754ad